In [2]:
import pytesseract
from pytesseract import Output
import csv
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob, os
import time

In [11]:
def get_column(check, word, start):
    global low_row
    word = word.strip()
    
    flag = False
    for i, check_line in enumerate(check[start:]):
        print(i, check_line)
    
        if check_line:
            try:
                if not word in check_line:
                    pass
                else:
                    fetched_pos = check_line.lower().find(word.lower())
                    #print("fetched pos ", fetched_pos)
                    flag = True
                   # print(i)
                    if fetched_pos < 16:
                        column = 0
                    elif fetched_pos < 33:
                        column = 1
                    elif fetched_pos < 52:
                        column = 2
                    elif fetched_pos < 67:
                        column = 3
                    elif fetched_pos < 108 and "@" in word:
                        column = 4
                    else:
                        column = 5 
                    
                    #print(f"{word} is in row: {low_row}, column: {column}")
                    return [i, column]
            except UnboundLocalError:
                print("THIS Could not find word.")
                return -1
    if not flag:
        print(f"Did not find \n {word}")
        return [flag,flag]
    else:
        low_row+= 1
        
def imshow(img, label='default'):
    cv2.imshow(label, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
path = input("Paste the folder path: ").replace("/","//")
all_images = glob.glob(os.path.join(path,"*.jpg"))

print(f"Found {len(all_images)}. Begin extraction")
data = pd.DataFrame()
for image in all_images:
    n = image.split("\\")[-1]
    print(f"{n}, ", end="")
    _s = time.time()
    
    img = cv2.imread(image)
    height, width, channel = img.shape
    imshow(img)


    img = cv2.resize(img, (width*2, height*2))
    imshow(img)
    alpha = 0.9
    beta = 0
    adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    adjusted = cv2.resize(adjusted, (width*2, height * 2))

    height, width, channel = img.shape


    # Convert image to grey scale, erode horizontally and draw horizontal line
    gray_image = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
    thresh_img = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    imshow(thresh_img)

    kernel = np.ones((1,30), np.uint8)  #for horizontal erosion
    e_im = cv2.erode(gray_image, kernel, iterations=2)  #performing erosion
    imshow(e_im)
    


    edges = cv2.Canny(e_im,0,150,apertureSize = 5)     #edges

    minLineLength=333
    #houghlines for detecting those erosion lines
    try:
        
        lines = cv2.HoughLinesP(image=edges,rho=0.5,theta=np.pi/180, threshold=222,lines=np.array([]), minLineLength=minLineLength,maxLineGap=7) 
        a,b,c = lines.shape
    except:
        pass
    # Draw the lines
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            cv2.line(thresh_img, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)

    imshow(thresh_img)

    #New kernel for Vertical erode and line
    kernel = np.ones((500,12), np.uint8)  #for horizontal erosion
    e_im = cv2.erode(gray_image, kernel, iterations=4)  #performing erosion
    imshow(e_im)

    edges = cv2.Canny(e_im,100,0,apertureSize = 3)
    imshow(edges)

    try:

        lines = cv2.HoughLinesP(image=edges,rho=0.5,theta=np.pi/180, threshold=222,lines=np.array([]), minLineLength=minLineLength,maxLineGap=7) 
        a,b,c = lines.shape
    except:
        pass
    # Draw the lines
    if lines is not None:
        for i in range(0, len(lines)):
            l = lines[i][0]
            cv2.line(thresh_img, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 3, cv2.LINE_AA)

    imshow(thresh_img)

    # Create computer vision

    custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'
    d = pytesseract.image_to_data(thresh_img, config=custom_config, output_type=Output.DICT)
    df = pd.DataFrame(d)

    # clean up blanks
    df1 = df[(df.conf != '-1') & (df.text != ' ') & (df.text != '')]
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)

    # sort blocks vertically
    sorted_blocks = df1.groupby('block_num').first().sort_values('top').index.tolist()
    for block in sorted_blocks:
        curr = df1[df1['block_num'] == block]
        sel = curr[curr.text.str.len() > 3]
        # sel = curr
        char_w = (sel.width / sel.text.str.len()).mean()
        prev_par, prev_line, prev_left = 0, 0, 0
        text = ''
        for ix, ln in curr.iterrows():

            # add new line when necessary
            if prev_par != ln['par_num']:
                text += '\n'
                prev_par = ln['par_num']
                prev_line = ln['line_num']
                prev_left = 0
            elif prev_line != ln['line_num']:
                text += '\n'
                prev_line = ln['line_num']
                prev_left = 0

            added = 0  # num of spaces that should be added
            if ln['left'] / char_w > prev_left + 1:
                added = int((ln['left']) / char_w) - prev_left
                text += ' ' * added
            text += ln['text'] + ' '
            prev_left += len(ln['text']) + added + 1

        text += '\n'
        #print(text)

        check = list(text.split("\n"))
        print(check)


    #Now read junk data


    output_text = []
    test = []
    custom_config = r'-l eng --oem 3 --psm 6 -c tessedit_char_whitelist="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789-:.$%./@& *"'


    # Get all OCR output information from pytesseract
    ocr_output_details = pytesseract.image_to_data(thresh_img, output_type = Output.DICT, config=custom_config, lang='eng')
    # Total bounding boxes
    n_boxes = len(ocr_output_details['text'])

    # Extract and draw rectangles for all bounding boxes
    for i in range(n_boxes):
        (x, y, w, h) = (ocr_output_details['left'][i], ocr_output_details['top'][i], ocr_output_details['width'][i], ocr_output_details['height'][i])
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Print OCR Output
    #print(ocr_output_details.keys())

    # Arrange output text from OCR into the format as per image

    word_list = []
    last_word = []

    for word in ocr_output_details['text']:
        # If there is no text in an element
        if word != '':
            word_list.append(word)
            last_word = word
        # Append final list of words with valid words
        if (last_word != '' and word == '') or (word == ocr_output_details['text'][-1]):
            output_text.append(word_list)
            test.append(word_list)
            word_list = []







    #Look through lens

    all_row = []
    low_row = 0
    for line in output_text:
        print(f"Line has {len(line)} words.")

        if line:
            print(line)
            row = [0 for x in range(7)]
            for idx, word in enumerate(line):
                print(idx,word)
                if word == '@' or word == '&' or word == "." or word == "@&":
                    continue
                pos = get_column(check, word, low_row)
                row_num = pos[0]
                column = pos[1]
                print(column)

                if not column and not column == 0 :
                    #low_row = low_row.append(line)
                    break

                elif type(row[column]) == int:
                    row[column] = word
                elif type(row[column]) == str:
                    row[column]+= ' '+word


            #print(row)
            row.append(row_num)
            all_row.append(row)

        else:
            pass
            #print(line)
    test = pd.DataFrame(all_row)
    test = pd.DataFrame(all_row)
    test.columns = ['Name','Company','Designation','Department','Email','Phone','Score','Row']
    test.iloc[:,:3] = test.iloc[:,:3].replace(0, np.nan)
    test['Name'].ffill(inplace =True)
    test['Row'] = image.split("\\")[-1]
    
    
    
    print(f"{str(round(time.time() - _s, 2))} seconds.")
    data = pd.concat([data,test])
    break


Paste the folder path: C:\Users\abhis\Desktop\shelf\Data Extraction - Images\Payload\Kolkatta 17Jan23
Found 93. Begin extraction
1.JPG, ['', ' @   Mr AK       Allahabad       GM  General         IT              gmit@allahabadbank.in             9004046079 ', ' Pandeya         Bank            Manager ', ' @   Mr Ronti    andhan   Rant      O.CIO            IT              ronti.kar@bandhanbank.com         9674175631 ', ' @   Abhishek    Paharpur        Manager             IT              a.datta@paharpur.com              9830365458 ', ' Datta           Cooling         Manager  IT ', '                 Towers Ltd ', ' Debasish Pal    Eveready        Manager             IT              debasishpal@eveready.co.in        9830133969 ', '                 Industries      Manager  - IT ', '                 India Ltd. ', ' Amit Roy        Eveready        Manager             IT              amitrc @eveready.co.in            9874178374 ', ' Choudhury       Industries      Manager  - IT ', '        

In [14]:
data

,Name,Company,Designation,Department,Email,Phone,Score,Row
0,Mr AK,Allahabad,GM General,IT,gmit@allahabadbank.in,9004046079,0,1.JPG
1,Pandeya,Bank,Manager,0,0,0,0,1.JPG
2,Mr Ronti,andhan Rant,O.CIO,IT,ronti.kar@bandhanbank.com,9674175631,0,1.JPG
3,Abhishek,Paharpur,Manager,IT,a.datta@paharpur.com,9830365458,0,1.JPG
4,Datta,Cooling,Manager,IT,0,0,0,1.JPG
5,Datta,Towers Ltd,NaN,0,0,0,0,1.JPG
6,Debasish Pal,Eveready,Manager,IT,debasishpal@eveready.co.in,9830133969,0,1.JPG
7,Debasish Pal,Industries,Manager -,IT,0,0,0,1.JPG
8,Debasish Pal,India Ltd.,NaN,0,0,0,0,1.JPG
9,Amit Roy,Eveready,Manager,IT,@eveready.co.in,amitrc 9874178374,0,1.JPG


In [70]:
data.to_excel("Kolkatta.xlsx")

In [ ]:
C:\Users\abhis\Desktop\shelf\Data Extraction - Images\Payload\Kolkatta 17Jan23